In [ ]:
import pyrootutils

root = pyrootutils.setup_root(__file__, indicator=".project-root", pythonpath=True)
root

In [ ]:
import torch
from torch import nn
from torchvision import models

densenet = models.densenet169(weights=models.DenseNet169_Weights)

print(densenet.features.conv0)
print(densenet.features.conv0.weight.size())
print(densenet.classifier)
print(densenet.classifier.weight.size())

conv = nn.Conv2d(
    in_channels=1,
    out_channels=densenet.features.conv0.out_channels,
    kernel_size=densenet.features.conv0.kernel_size,
    stride=densenet.features.conv0.stride,
    padding=densenet.features.conv0.padding,
    dilation=densenet.features.conv0.dilation,
    groups=densenet.features.conv0.groups,
    bias=densenet.features.conv0.bias is not None,
    padding_mode=densenet.features.conv0.padding_mode,
)
conv.weight = nn.Parameter(torch.mean(densenet.features.conv0.weight, 1, True))
densenet.features.conv0 = conv

densenet.classifier = nn.Linear(
    in_features=densenet.classifier.in_features,
    out_features=6,
)

print("-----")
print(densenet.features.conv0)
print(densenet.features.conv0.weight.size())
print(densenet.classifier)
print(densenet.classifier.weight.size())

In [ ]:
from src.datamodules.fetal_planes import FetalPlanesDataModule

dm = FetalPlanesDataModule(data_dir=root / "data", batch_size=32)
dm.prepare_data()
dm.setup()
batch = next(iter(dm.train_dataloader()))
x, y = batch

densenet(x).size()